In [15]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split


from model_model import SiameseNetwork
from model_configs import ModelDimConfigs, TrainingConfigs
from misc_tools import get_timestamp

In [16]:
import hiddenlayer as h

In [17]:
dimconf = ModelDimConfigs(
        rnn_in_size=64, 
        lin_in_size_1=32, 
        lin_in_size_2=16, 
        lin_out_size_2=1
    )

model = SiameseNetwork(
    dimconf=dimconf
)